# Description

This notebook demonstrates how to use the NVIDIA IndeX WebSocket API to connect to a running session and load your own dataset.

Let's dive in.

## Pre-requisites

You will require the `websocket-client` python package installed into your environment.

You also need to make sure that the `nvindex_util.py` file is available in same path as the notebook.

In [ ]:
import time
import base64
import subprocess
import os

from matplotlib import colors
from matplotlib.ticker import PercentFormatter

from IPython.display import IFrame, Image

try:
    from websocket import create_connection
except:
    raise Exeption("websocket-client package is missing")

try:
    import nvindex_util
except:
    raise Exception("nvindex_util.py is missing from your path.")

## Connecting to a running NVIDIA IndeX session

Assuming you have a running session, you can just plug it in under in the `TODO` string. You can also use the `SERVICE_ENDPOINT` environment variable

Examples of endpoint urls for a running session would be:
- `http://localhost:8080`
- `https://nvindex.app.cloud.example.com`

Next, you also have to fill in the credentials.

In [ ]:
# FILL IN SESSION URL
service=os.getenv("SERVICE_ENDPOINT", None)
# FILL IN YOUR PASSWORD
user='nvindex'
password='test123'

if service == None or password == None:
    raise Exception("Session URL and/or password are missing.")

In [ ]:
# get WebSocket endpoint url: http -> ws (just for brevity)
service_ws=service.replace("http", "ws")
print(service)
print(service_ws)

## Connecting the Notebook to the NVIDIA IndeX session 

Now we can establish a connection to the server to receive the NVIDIA IndeX viewer and embed the viewer in the notebook. In this demo the render service delivers the NVIDIA IndeX HTML5 viewer with extended user-interface. Generally, this render service can be configured to deliver the interactive video stream or single images only.

In [ ]:
IFrame(f"{service}", width=980, height=650)

## Interacting with the WebSocket API

As you can see, a default (synthetic) volume is already loaded. Let's use the WebSocket API to clear the volume and it's associated scene. To do that, we need to connect to the command WebSocket url for sending commands:

In [ ]:
ws_command_url = f"{service_ws}/index_command_client"
print(f"Command WebSocket url: f{ws_command_url}")
ws = nvindex_util.get_websocket(ws_command_url, credentials=(user, password))

WebSockets are used by the NVIDIA IndeX viewer in 2 ways:
- The H.264 encoded video stream is sent over a WebSocket to the browser.
- The command WebSocket is used to control the viewer by issuing JSON RPC like commands.

The simplest way to clear the volume is to clear the whole scene graph. Let's send a JSON RPC command to do just that:

In [ ]:
# Send the command to fetch all scene elements
ok, ret = nvindex_util.send_jsonrpc_command(ws, {
  "method": "nv::index::app::scene_management_center::scene_management_center_command_receiver.get_scene_graph",
  "params": {
      "scene_graph_representation_type": "tree"                              
  }                          
})

assert(ok is True)

# Now clear each individual element
children = ret.get('result', {}).get('tree', {}).get('children', [])
for ch in children:                
  ok, ret = nvindex_util.send_jsonrpc_command(ws, {
      "method": "nv::index::app::scene_management_center::scene_management_center_command_receiver.set_scene_element",
      "params": {
          "id": "scene_root",                                                                                            
          "args": { 
              "remove.id": ch['id']   
          }                                              
      }                                                         
   }) 

Now we should see a black canvas with an empty scene:

In [ ]:
IFrame(f"{service}", width=980, height=650)

## Loading a raw volume dataset

Now we are ready to load a dataset. We will be using the sample Supernova dataset (for more information see our [sample dataset page](https://github.com/NVIDIA/nvindex-cloud/blob/master/doc/datasets.md). The dataset has an extent of 633 voxels on each axis.

IndeX can load data from local disk, from cloud storage (Google Storage, AWS S3, Azure Blob Storage) or directly over HTTP/HTTPS as long as the server supports byte ranges. We will use the latter option for simplicity: downloading from S3 via HTTP.

To add a dataset from scratch, we have to set up a scene that contains at least the following scene elements:
a colormap and a volume scene element.

In [ ]:
data_path = "https://nvindex-datasets-us-west2.s3.us-west-2.amazonaws.com/scenes/00-supernova_ncsa_small/data/Export_entropy_633x633x633_uint8_T1074.raw"
bbox = "0 0 0 633 633 633"
colormap_path = "https://nvindex-datasets-us-west2.s3.us-west-2.amazonaws.com/scenes/00-supernova_ncsa_small/scene/colormaps/nova03.cmap"

ok, ret = nvindex_util.send_jsonrpc_command(ws, {
  "method": "nv::index::app::scene_management_center::scene_management_center_command_receiver.create_scene_element",
  "params": {
      "parent_group_id": "scene_root",
      "new_scene_element_id": f"my_colormap",
      "class_name": "IColormap",
      "args": {
          "map_type": "data",
          "data_source": "file",
          "file_type": "cmap",
          "file_name": colormap_path,
          "domain_boundary_mode": "clamp_to_edge",
      }
  }
})

ok, ret = nvindex_util.send_jsonrpc_command(ws, {
  "method": "nv::index::app::scene_management_center::scene_management_center_command_receiver.create_scene_element",
  "params": {
      "parent_group_id": "scene_root",
      "new_scene_element_id": "my_volume",
      "class_name": "ISparse_volume_scene_element",
      "args":  {
          "type": "sparse_volume",
          "bbox": bbox,
          "importer": "nv::index::plugin::base_importer.Sparse_volume_importer_raw",
          "input_file": data_path,
          "convert_zyx_to_xyz": "false",
          "voxel_format": "uint8",
      }
  }
})


Now we have to wait for the data to be loaded...

In [ ]:
nvindex_util.wait_for_data_to_load(ws)
IFrame(f"{service}", width=980, height=650)